In [1]:
pip freeze > requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [3]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
import pandas as pd
import numpy as np
from datetime import datetime


In [4]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///loan_calculator.db'
db = SQLAlchemy(app)


In [5]:
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), nullable=False)
    income = db.Column(db.Float, nullable=False)

class Loan(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    amount = db.Column(db.Float, nullable=False)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    date_borrowed = db.Column(db.Date, nullable=False)


In [6]:
with app.app_context():
    db.create_all()


In [7]:
@app.route('/user', methods=['POST'])
def add_user():
    data = request.json
    new_user = User(name=data['name'], income=data['income'])
    db.session.add(new_user)
    db.session.commit()
    return jsonify({'message': 'User added successfully'}), 201


In [8]:
@app.route('/loan', methods=['POST'])
def add_loan():
    data = request.json
    new_loan = Loan(amount=data['amount'], user_id=data['user_id'], date_borrowed=datetime.strptime(data['date_borrowed'], '%Y-%m-%d'))
    db.session.add(new_loan)
    db.session.commit()
    return jsonify({'message': 'Loan added successfully'}), 201


In [9]:
@app.route('/total_borrowed', methods=['GET'])
def total_borrowed():
    total = db.session.query(db.func.sum(Loan.amount)).scalar()
    return jsonify({'total_borrowed': total}), 200


In [10]:
@app.route('/average_income', methods=['GET'])
def average_income():
    avg_income = db.session.query(db.func.avg(User.income)).scalar()
    return jsonify({'average_income': avg_income}), 200


In [11]:
@app.route('/highest_earning_month', methods=['GET'])
def highest_earning_month():
    loans = db.session.query(Loan).all()
    df = pd.DataFrame([(loan.amount, loan.date_borrowed) for loan in loans], columns=['amount', 'date_borrowed'])
    df['year_month'] = df['date_borrowed'].apply(lambda x: x.strftime('%Y-%m'))
    highest_month = df.groupby('year_month')['amount'].sum().idxmax()
    return jsonify({'highest_earning_month': highest_month}), 200


In [12]:
@app.route('/predicted_income', methods=['GET'])
def predicted_income():
    loans = db.session.query(Loan).all()
    df = pd.DataFrame([(loan.amount, loan.date_borrowed) for loan in loans], columns=['amount', 'date_borrowed'])
    df['date_borrowed'] = pd.to_datetime(df['date_borrowed'])
    df.set_index('date_borrowed', inplace=True)
    monthly_data = df.resample('M').sum()
    
    # Prepare data for regression model
    monthly_data['month'] = monthly_data.index.month
    monthly_data['year'] = monthly_data.index.year
    X = monthly_data[['year', 'month']]
    y = monthly_data['amount']
    
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    model.fit(X, y)
    
    future_months = pd.date_range(start=df.index.max(), periods=3, freq='M')
    future_data = pd.DataFrame({'year': future_months.year, 'month': future_months.month})
    predictions = model.predict(future_data)
    
    return jsonify({'predicted_income': predictions.tolist()}), 200


In [13]:
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\elich\Documents\Loan Calculator\venv\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
